<a href="https://colab.research.google.com/github/SubbulakshmiSN/Multi_Class_Text_Classification/blob/main/Subbulakshmi_S_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Multi Class Text Classification

# Introduction

The project involves working with a multi-label classification problem in a structured data environment. The goal is to predict multiple labels for given inputs, which is common in various domains such as healthcare, finance, and marketing. Multi-label classification helps in understanding and predicting complex relationships within the data.

# Problem Statement
The objective is to predict multiple target labels for each record in the dataset using a model that handles high cardinality features efficiently.



## *Imports*

In [ ]:
!pip install datatable
import datatable as dt
from datatable.models import Ftrl
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

## *Data Loading and Exploration:*
Loaded the training, test, and label data using __datatable__ and conducted an initial exploration to understand the data structure.

In [ ]:
train=dt.fread('train.csv')
test=dt.fread('test_new.csv')
trainlabels=dt.fread('trainLabels.csv')

In [ ]:
train.head(5)

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,…,x141,x142,x143,x144,x145
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.0731394,0.481394,0.115697,0.472474,…,NO,YES,2,0.375535,0.46461
1,2,,,,,0,0,0,0,0,…,NO,YES,4,0.741682,0.59363
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.3418,0.0514217,0.935572,0.0414398,0.50171,…,NO,NO,1,0.776467,0.493159
3,4,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.0414707,0.940787,0.0908508,0.556564,…,NO,YES,3,0.168234,0.546582
4,5,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.41592,0,1,0,0.375297,…,NO,NO,1,0.246637,0.361045


In [ ]:
test.head(5)

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,…,x141,x142,x143,x144,x145
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.4148,0,1,0,0.20206,…,NO,NO,0,0.0896861,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.0498339,0.945938,0.317427,0.482021,…,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.41592,0,1,0,0.703088,…,NO,NO,8,0.557175,0.693587
3,1698004,,,,,0,0,0,0,0,…,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.12921,0.0870202,0.81424,1.1128,0.874318,…,NO,YES,2,0.224729,0.870909


In [ ]:
trainlabels.head(5)

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,…,y29,y30,y31,y32,y33
,▪▪▪▪,▪,▪,▪,▪,▪,▪,▪,▪,▪,,▪,▪,▪,▪,▪
0,1,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,…,0,0,0,1,0
2,3,0,0,1,0,0,0,0,0,0,…,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1


In [ ]:
print(train.shape,test.shape,trainlabels.shape)

(9999, 146) (2000, 146) (49999, 34)


## *Data Merging*

Merged the training data with the labels based on the __id__ column to create a comprehensive dataset for model training.

In [ ]:
 #Set keys for joining
train.key = "id"
trainlabels.key = "id"

In [ ]:
train_merged = train[:, :, dt.join(trainlabels)]

In [ ]:
print(train_merged.shape)
train_merged.head(5)

(9999, 179)


,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,…,y29,y30,y31,y32,y33
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪,▪,▪,▪,▪
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.0731394,0.481394,0.115697,0.472474,…,0,0,0,0,1
1,2,,,,,0,0,0,0,0,…,0,0,0,1,0
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.3418,0.0514217,0.935572,0.0414398,0.50171,…,0,0,0,0,0
3,4,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.0414707,0.940787,0.0908508,0.556564,…,0,0,0,0,1
4,5,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.41592,0,1,0,0.375297,…,0,0,0,0,1


## *Data Preparation*

Split the merged data into features (X_train) and labels (y_train) and prepared the test data (X_test) for prediction.

In [ ]:
X_train = train_merged[:, 1:-(len(trainlabels.names) - 1)]  # All columns except the ID and label columns
y_train = train_merged[:, -(len(trainlabels.names) - 1):]  # All columns corresponding to the labels


In [ ]:
print(X_train.shape, y_train.shape)


(9999, 145) (9999, 33)


In [ ]:
X_train.head(5)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,…,x141,x142,x143,x144,x145
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.0731394,0.481394,0.115697,0.472474,YES,…,NO,YES,2,0.375535,0.46461
1,,,,,0,0,0,0,0,,…,NO,YES,4,0.741682,0.59363
2,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.3418,0.0514217,0.935572,0.0414398,0.50171,NO,…,NO,NO,1,0.776467,0.493159
3,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.0414707,0.940787,0.0908508,0.556564,YES,…,NO,YES,3,0.168234,0.546582
4,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.41592,0,1,0,0.375297,NO,…,NO,NO,1,0.246637,0.361045


In [ ]:
y_train.head(5)

,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,…,y29,y30,y31,y32,y33
,▪,▪,▪,▪,▪,▪,▪,▪,▪,▪,,▪,▪,▪,▪,▪
0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,…,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,…,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1


In [ ]:
test_ids = test[:, "id"]
X_test = test[:, 1:]

## *Model Training and Prediction*

 *Trained an FTRL model for each target label and made predictions on the test data.*


 __FTRL Model__: Chosen for its ability to handle large datasets with high cardinality features efficiently. The Follow-The-Regularized-Leader (FTRL) model combines L1 and L2 regularization, making it suitable for sparse datasets. It is selected due to its effectiveness in managing high-dimensional feature space and providing probabilistic outputs for each label

In [ ]:
submission = dt.Frame()

# Train and predict for each target
for target in y_train.names:
    print(f'Model for target {target}')

    model_ftrl = Ftrl(nepochs=5, nbins=10**8, lambda1=0.1)
    model_ftrl.fit(X_train, y_train[target])

    preds_test = model_ftrl.predict(X_test)
    submission_target = dt.Frame(id_label=test_ids[:, dt.as_type(dt.f.id, str) + f'_{target}'],
                                 pred=preds_test['True'])
    submission.rbind(submission_target)

Model for target y1
Model for target y2
Model for target y3
Model for target y4
Model for target y5
Model for target y6
Model for target y7
Model for target y8
Model for target y9
Model for target y10
Model for target y11
Model for target y12
Model for target y13
Model for target y14
Model for target y15
Model for target y16
Model for target y17
Model for target y18
Model for target y19
Model for target y20
Model for target y21
Model for target y22
Model for target y23
Model for target y24
Model for target y25
Model for target y26
Model for target y27
Model for target y28
Model for target y29
Model for target y30
Model for target y31
Model for target y32
Model for target y33


## *Final Submission*

Created a submission file with the predicted probabilities for each target label.

In [ ]:
# Save submission
submission.to_csv('submission.csv')

In [ ]:
submission=pd.read_csv('submission.csv')

In [ ]:
submission

,id_label,pred
0,1698001_y1,0.001717
1,1698002_y1,0.001951
2,1698003_y1,0.002069
3,1698004_y1,0.020571
4,1698005_y1,0.002927
...,...,...
65995,1699996_y33,0.423708
65996,1699997_y33,0.344908
65997,1699998_y33,0.888941
65998,1699999_y33,0.817318


## *Report*

***Overview***

  In this project, developed a predictive model using the __Follow-the-Regularized-Leader (FTRL)__ algorithm to classify multiple targets from a given dataset and  leveraged the __datatable__ library for efficient data handling and processing, which is particularly beneficial when working with large datasets.


### *Hash Handling*

The __FTRL algorithm__ and the __datatable__ library handle __hash__ encoding internally. The Ftrl model uses a hashing trick to efficiently map features into a fixed-size hash space, which helps in handling high-dimensional data and prevents overfitting. This approach is particularly useful for models with sparse data and a large number of categorical features.

In this project, we didn't explicitly preprocess or encode features as the FTRL model inherently supports hashing for feature representation. This allowed us to focus on the overall workflow without worrying about feature engineering complexities.

In order to treat __hash__ columns we can use __hashlib__ library which converts the hashes into numbers.